In [ ]:
MARKET = 'NSE'
import pandas as pd

df_unds = pd.read_pickle('./data/df_'+MARKET.lower()+'_unds.pkl')
df_ohlcs = pd.read_pickle('./data/df_'+MARKET.lower()+'_ohlcs.pkl')
df_chains = pd.read_pickle('./data/df_'+MARKET.lower()+'_chains.pkl')
df_symlots = pd.read_pickle('./data/'+MARKET.lower()+'_symlots.pkl')
df_raw_opts = pd.read_pickle('./data/df_' + MARKET.lower() + '_qopts.pkl')
df_margins = pd.read_pickle('./data/df_'+MARKET.lower()+'_opt_margins.pkl')
df_prices = pd.read_pickle('./data/df_'+MARKET.lower()+'_opt_prices.pkl')

In [ ]:
# determine the lots
# Can be ignored in base - as df_opts already has the lots

if MARKET == 'NSE':
    df_opts['expiryM'] = df_opts.expiry.apply(
        lambda d: d[:4] + '-' + d[4:6])
    cols1 = ['symbol', 'expiryM']
    df_opts = df_opts.set_index(cols1).join(
        df_symlots[cols1 + ['lot']].set_index(cols1)).reset_index()
    df_opts = df_opts.drop('expiryM', 1)
else:
    df_opts['lot'] = 100

In [ ]:
from support import get_dte, calcsd_df
import numpy as np

# merge price, margins and lots
cols_to_use = list(df_margins.columns.difference(df_prices.columns)) + ['conId']
df = df_prices.set_index('conId').join(df_margins[cols_to_use].set_index('conId')).reset_index().drop('secType', 1)
df = df.set_index('conId').join(df_opts.set_index('conId').lot).reset_index()

# add intrinsic and time values
df = df.assign(intrinsic=np.where(df.right == 'C', 
                           (df.undPrice-df.strike).clip(0, None), 
                           (df.strike-df.undPrice).clip(0, None)))
df = df.assign(timevalue=df.price - df.intrinsic,
                         dte=df.expiry.apply(get_dte))

# ... compute std deviation
df['stdev'] = calcsd_df(df.strike, df)
df['rom'] = (df.price*df.lot-df.comm).clip(0)/df.margin*365/df.dte

In [ ]:
df[df.bid > 0].sort_values('rom', ascending=False).head(10)

In [ ]:

from ib_insync import *
cos = [(c, o) for c, o in zip(df_opts.contract, np.repeat(MarketOrder('SELL', 11),len(df_opts)))][:5]

In [ ]:
[o.totalQuantity if ct.exchange == 'NSE' else 100 for ct, o in cos]

In [ ]:
cos